In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc,html,Input, Output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,precision_recall_curve,confusion_matrix,ConfusionMatrixDisplay
import seaborn as sns
from ipywidgets import interact,widgets, Layout


In [2]:

percent_of_criminal =np.array([0.2,0.25,0.3,0.4,0.6,0.65,0.68,0.8,0.82,0.91,0.93,0.96])
criminal = np.array([0,0,0,0,1,0,1,1,0,1,1,1])


In [3]:
fpr,tpr,threshold = roc_curve(y_true=criminal,y_score=percent_of_criminal)

In [4]:
def ROC(thr) : 
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(8,8))

    ax.plot(fpr,tpr, linewidth=2)
    plt.xlabel('FP rate',fontsize=18)
    plt.ylabel('TP rate',fontsize=18)
    plt.xlim(-0.02,1.01)
    plt.ylim(-0.02,1.01)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    #점표현
    ax.plot([fpr[thr]],[tpr[thr]],'ro')
    # plt.arrow(x=0.7, y=0.3, dx=(fpr[thr]-0.67), dy=(tpr[thr]-0.295), width=0.003, facecolor='red',alpha=0.5, edgecolor='none') 
    plt.annotate('-', xy=(fpr[thr],tpr[thr]), xytext=(0.65,0.3),
            fontsize=14, ha='center',
            arrowprops=dict(facecolor='red', alpha=0.5, width=1.5, shrink=0.1, headwidth=7))
    
    sns.set(font_scale=1.2)
    ax2 = fig.add_axes([0.55, 0.2, 0.25, 0.3])
    pred_log = (percent_of_criminal >= threshold[thr])
    con_max = confusion_matrix(criminal,pred_log,labels=[1, 0])
    ConfusionMatrixDisplay(con_max).plot(colorbar=False,cmap='PuBu',ax=ax2)
    ax2.xaxis.set_label_position('top') 
    ax2.xaxis.tick_top()
    ax2.grid(False)
    plt.xticks([1,0],['Negative','Positive'],fontsize=12)
    plt.yticks([1,0],['False','True'],fontsize=12)
    return fig

In [11]:
%matplotlib inline

slider = widgets.IntSlider(min=0,max=7,step=1, value=0, layout=Layout(width='500px'))
# slider = widgets.FloatSlider(min=1,max=0,step=0.125, value=0, layout=Layout(width='500px'))
slider.style.handle_color = 'lightblue'
interact(ROC, thr=slider)

interactive(children=(IntSlider(value=0, description='thr', layout=Layout(width='500px'), max=7, style=SliderS…

<function __main__.ROC(thr)>

In [7]:
import inspect
inspect.getfullargspec(ff.create_annotated_heatmap)

FullArgSpec(args=['z', 'x', 'y', 'annotation_text', 'colorscale', 'font_colors', 'showscale', 'reversescale'], varargs=None, varkw='kwargs', defaults=(None, None, None, 'Plasma', None, False, False), kwonlyargs=[], kwonlydefaults=None, annotations={})

In [15]:
import plotly.figure_factory as ff

thr = 0
pred_log = (percent_of_criminal >= threshold[thr])
con_max = confusion_matrix(criminal,pred_log,labels=[1, 0])

fig_3 = ff.create_annotated_heatmap(con_max,x=['Positive','Negative'], y=['False','True'])
fig_3.update_traces(dict(showscale=False, 
                    coloraxis=None, 
                    colorscale='Viridis'), selector={'type':'heatmap'})
fig.show()

In [12]:
thr = 0
pred_log = (percent_of_criminal >= threshold[thr])
con_max = confusion_matrix(criminal,pred_log,labels=[1, 0])
fig = px.imshow(con_max, color_continuous_scale='BuPu',width=400,height=400, labels=dict(x='Predict Labels', y='True Labels'),x=['Positive','Negative'], y=['True','False'])
fig.update_xaxes(side='top')
fig.show()

(Heatmap({
     'coloraxis': 'coloraxis',
     'hovertemplate': 'Predict Labels: %{x}<br>True Labels: %{y}<br>color: %{z}<extra></extra>',
     'name': '0',
     'x': [Positive, Negative],
     'xaxis': 'x',
     'y': [True, False],
     'yaxis': 'y',
     'z': array([[0, 6],
                 [0, 6]], dtype=int64)
 }),)

In [31]:
thr=2
# subplots
left_width = 0.7
# fig = make_subplots(rows=1, cols=2,column_widths=[left_width, 1 - left_width])
fig = make_subplots(rows=3, cols=3,specs=[[{"colspan": 2, 'rowspan' : 3},None, None],[None,None,{}],[None,None,None]])
# subplot 1번에 넣기
fig_2 = px.line(x=fpr,y=tpr)
fig.append_trace(fig_2.data[0],row=1,col=1)

# subplot 1번에 있는 plot과 합치기

# base classifier
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False),row=1,col=1)
# dot
fig.add_trace(
    go.Scatter(
        x=[fpr[thr]],
        y=[tpr[thr]],
        showlegend=False,
        marker=dict(color="red", size=10)),
        row=1,col=1)

# subplot 2번에 넣기
pred_log = (percent_of_criminal >= threshold[thr])
con_max = confusion_matrix(criminal,pred_log,labels=[1, 0])
fig_3 = px.imshow(con_max, text_auto=True ,color_continuous_scale='BuPu',x=['Positive','Negative'], y=['False','True'])
fig_3.update_traces(dict(showscale=False, 
                    coloraxis=None, 
                    colorscale='BuPu'), selector={'type':'heatmap'})
fig_3.update_xaxes(side='top')

fig_3.update_xaxes(side='top')
fig.add_trace(fig_3.data[0],row=2,col=3)

# plot update
fig.update_layout(width=800,height=600)
fig.update_xaxes(title_text="Predict Value",row=2,col=3,side='top')
fig.update_yaxes(title_text="True Value",row=2,col=3)


In [99]:

app = JupyterDash(__name__)

#marker
# marker = {num : str(i) for num,i in enumerate(threshold)}
marker = {0: '100%', 1: '96%', 2: '91%', 3: '82%', 4: '68%', 5: '65%', 6: '60%', 7: '0%'}
# app.layout =html.Div([
#     html.H1('ROC_Curve', style={'text-align': 'center'}),
#     # html.Iframe(id='curve', srcDoc=None,style={'width' : '100%', 'height' : '500px' }),
#     html.Div([
#         "Threshold",
#         dcc.Slider(0, 7, 1, value=0, id='thr', marks=marker)],
#         # style=dict(width=600),
#         style = {"width": "2%", "height":"100%","display":"inline-block","position":"relative"}            
#         ),
#     dcc.Graph(id='curve'),
#     # html.Div(["색상",dcc.RadioItems(id='color', value='aliceblue', options=c_codes['0'], className='btn-group', style={'flex-wrap' : 'wrap'},labelClassName= 'btn-outline-secondary',labelStyle={'text-align' : 'left', 'width' : 180, 'height' : 25  } ,inputStyle={'margin-right': 5})],style={'margin': 20}) #inline=True,
# ])
app.layout=html.Div([
    html.H1('ROC_Curve', style={'text-align': 'center'}),
    html.Div([
        html.Div([dcc.Slider(0, 7, 1, value=0, id='thr', marks=marker,vertical=True)],style = {"width": "7%", "height":"100%","display":"inline-block","position":"relative",'text-align' : 'center'},className='col'),
        dcc.Graph(id='curve',className='col-md-auto'),
        html.Div('',className='col')
    ],className='row row-cols-auto',style={'justify-content': 'center','align-items' : 'center'})
],className='container')

@app.callback(
    Output(component_id='curve',component_property='figure'),
    Input('thr','value') # Input('location','value'),
)


def roc_df(thr) :
    # subplots
    fig = make_subplots(rows=3, cols=3,specs=[[{"colspan": 2, 'rowspan' : 3},None, None],[None,None,{}],[None,None,None]])
    # subplot 1번에 넣기
    fig_2 = px.line(x=fpr,y=tpr)
    fig.append_trace(fig_2.data[0],row=1,col=1)

    # subplot 1번에 있는 plot과 합치기

    # base classifier
    fig.add_trace(
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode="lines",
            line=go.scatter.Line(color="gray"),
            showlegend=False),row=1,col=1)
    # dot
    fig.add_trace(
        go.Scatter(
            x=[fpr[thr]],
            y=[tpr[thr]],
            showlegend=False,
            marker=dict(color="red", size=10)),
            row=1,col=1)

    # subplot 2번에 넣기
    pred_log = (percent_of_criminal >= threshold[thr])
    con_max = confusion_matrix(criminal,pred_log,labels=[1, 0])
    fig_3 = px.imshow(con_max, text_auto=True ,color_continuous_scale='BuPu',x=['Positive','Negative'], y=['True','False'])
    fig_3.update_traces(dict(showscale=False, 
                        coloraxis=None, 
                        colorscale='BuPu'), selector={'type':'heatmap'})
    fig_3.update_xaxes(side='top')
    fig.add_trace(fig_3.data[0],row=2,col=3)

    # plot update
    fig.update_layout(width=800,height=600)
    fig.update_xaxes(title_text="Predict Value",row=2,col=3,side='top')
    fig.update_yaxes(title_text="True Value",row=2,col=3,autorange='reversed')
    fig.update_xaxes(title_text='FP Rate',range=[-0.02,1.02], row=1,col=1)
    fig.update_yaxes(title_text='TP Rate',range=[-0.02,1.02], row=1,col=1)

    return fig


app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


In [103]:
b_suspect =[0.2,0.35,0.37,0.45,0.49,0.55,0.66,0.78,0.85,0.88,0.95,0.99]
b_criminal = [0,0,0,0,0,1,0,1,1,1,1,1]

c_suspect =[0.4,0.5,0.6,0.66,0.7,0.81,0.83,0.85,0.93,0.91,0.93,0.97]
c_criminal = [0,0,1,0,1,0,1,0,1,1,0,1]

fpr_b,tpr_b,threshold_b = roc_curve(b_criminal,b_suspect)
fpr_c,tpr_c,threshold_c = roc_curve(c_criminal,c_suspect)

In [141]:
# subplots
# subplot 1번에 넣기
fig = px.line(x=fpr,y=tpr)
fig.update_traces(line_color='darkgreen',opacity=0.5,name='Team_A',showlegend=True)
fig_2 = px.line(x=fpr_b,y=tpr_b)
fig_2.update_traces(line_color='orange',opacity=0.5,name='Team_B',showlegend=True)
fig.add_trace(fig_2.data[0])
fig_3 = px.line(x=fpr_c,y=tpr_c)
fig_3.update_traces(line_color='darkmagenta',opacity=0.5,name='Team_C',showlegend=True)
fig.add_trace(fig_3.data[0])
fig_4 = px.line(x=[0,1],y=[0,1])
fig_4.update_traces(line=dict(dash="dot", width=2),line_color='gray',opacity=0.5,name='Base_rate',showlegend=True)
fig.add_trace(fig_4.data[0])


# plot update
fig.update_layout(width=600,height=600)
fig.update_xaxes(title_text='FP Rate',range=[-0.02,1.02])
fig.update_yaxes(title_text='TP Rate',range=[-0.02,1.02])

fig.show()